## Imports and credentials

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

with open('/Users/dusti/.secret/yelp_api.json') as f: 
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [2]:
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

yelp_api

## Define search

In [3]:
LOCATION = 'Champaign, IL'
TERM = 'sushi'

In [4]:
JSON_FILE = "Data/results_in_progress_Champaign_sushi.json"
JSON_FILE

'Data/results_in_progress_Champaign_sushi.json'

In [5]:
file_exists = os.path.isfile(JSON_FILE)

if file_exists == False:
    

    folder = os.path.dirname(JSON_FILE)
    
    if len(folder)>0:
        
        os.makedirs(folder,exist_ok=True)
        
        
    
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  

else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_Champaign_sushi.json not found. Saving empty list to file.


## Determine how many results are already in the file

In [6]:
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    

n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


## Determine how many pages needed

In [7]:
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

## Delete previous results file

In [8]:
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)

False

## Re-run code with new function

In [9]:
import os
import json

def create_json_file(JSON_FILE, delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)

In [10]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_Champaign_sushi.json not found. Saving empty list to new file.
- 0 previous results found.


2

In [11]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/2 [00:00<?, ?it/s]

## Convert .json to df

In [12]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,ml2qTQ3qlgrTN2W2VvdKmQ,sushi-siam-champaign,Sushi Siam,https://s3-media2.fl.yelpcdn.com/bphoto/4Ak0PS...,False,https://www.yelp.com/biz/sushi-siam-champaign?...,41,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",4.5,"{'latitude': 40.09733255075801, 'longitude': -...","[delivery, pickup]","{'address1': '1731 W Kirby Ave', 'address2': N...",+12176071906,(217) 607-1906,1776.101282,NaN
1,-zEpEmDfFQL-ph0N3BDlXA,sakanaya-champaign-2,Sakanaya,https://s3-media2.fl.yelpcdn.com/bphoto/cHogr6...,False,https://www.yelp.com/biz/sakanaya-champaign-2?...,593,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 40.11001, 'longitude': -88.23307}","[delivery, pickup]","{'address1': '403 E Green St', 'address2': Non...",+12173441400,(217) 344-1400,3780.984404,$$
2,avyN1FdXcxQfrM8Mqz7WGA,ishi-sushi-and-kaori-s-oven-champaign,ISHI sushi & Kaori’s Oven,https://s3-media1.fl.yelpcdn.com/bphoto/cSZUki...,False,https://www.yelp.com/biz/ishi-sushi-and-kaori-...,6,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",5.0,"{'latitude': 40.09874, 'longitude': -88.24724}",[],"{'address1': '305 Cedar St', 'address2': '', '...",,,3028.885859,NaN
3,NEVA0IYbawceL6kz5v5DAw,kofusion-champaign,Kofusion,https://s3-media2.fl.yelpcdn.com/bphoto/a20kCP...,False,https://www.yelp.com/biz/kofusion-champaign?ad...,358,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",3.5,"{'latitude': 40.1176334856215, 'longitude': -8...",[delivery],"{'address1': '30 E Main St', 'address2': None,...",+12175311166,(217) 531-1166,3004.482002,$$
4,0-WocGTpO3Zm4q1Zzz49Rw,sushi-kame-champaign,Sushi Kame,https://s3-media2.fl.yelpcdn.com/bphoto/p_1Q9W...,False,https://www.yelp.com/biz/sushi-kame-champaign?...,169,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",3.5,"{'latitude': 40.109781, 'longitude': -88.240886}",[delivery],"{'address1': '39 E Green St', 'address2': None...",+12173563366,(217) 356-3366,3111.976400,$$


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
30,t0f8qXrKGNWv6j8MGmosRg,caffé-bene-urbana-3,Caffé Bene,https://s3-media4.fl.yelpcdn.com/bphoto/IFqwV2...,False,https://www.yelp.com/biz/caff%C3%A9-bene-urban...,76,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3.0,"{'latitude': 40.106139, 'longitude': -88.221243}","[pickup, delivery]","{'address1': '700 S Gregory St', 'address2': '...",+12179178000,(217) 917-8000,4823.970165,$$
31,FeXsYKhyJ7F8t0-kn5JldQ,schnucks-urbana-urbana-2,Schnucks Urbana,https://s3-media3.fl.yelpcdn.com/bphoto/2pHvIY...,False,https://www.yelp.com/biz/schnucks-urbana-urban...,30,"[{'alias': 'grocery', 'title': 'Grocery'}]",3.0,"{'latitude': 40.1139865, 'longitude': -88.2033...",[],"{'address1': '200 N Vine St', 'address2': '', ...",+12173376016,(217) 337-6016,6284.314756,$$
32,IYASJOu_TXz8PpPbt-Clbg,sushi-san-urbana,Sushi San,,False,https://www.yelp.com/biz/sushi-san-urbana?adju...,4,"[{'alias': 'restaurants', 'title': 'Restaurant...",2.5,"{'latitude': 40.11025, 'longitude': -88.22715}",[],"{'address1': '1401 W Green St', 'address2': ''...",+12173281886,(217) 328-1886,4276.199996,$
33,0aA5l11oXes0aS6mkN5lMQ,szechuan-china-champaign,Szechuan China,https://s3-media2.fl.yelpcdn.com/bphoto/Oh-Vs6...,False,https://www.yelp.com/biz/szechuan-china-champa...,54,"[{'alias': 'szechuan', 'title': 'Szechuan'}]",2.5,"{'latitude': 40.1128, 'longitude': -88.239095}","[pickup, delivery]","{'address1': '401 S 1st St', 'address2': '', '...",+12173989999,(217) 398-9999,3234.115005,$$
34,U_94mqL35HOyfAbj2LADIg,the-poké-shack-urbana,The Poké Shack,,False,https://www.yelp.com/biz/the-pok%C3%A9-shack-u...,2,"[{'alias': 'poke', 'title': 'Poke'}, {'alias':...",3.0,"{'latitude': 40.11519, 'longitude': -88.20804}",[],"{'address1': '401 N Broadway Ave', 'address2':...",,,5875.524751,NaN


## Check for duplicates

In [13]:
final_df.duplicated(subset='id').sum()

0

In [14]:
final_df.to_csv('Data/final_results_Champaign_Sushi.csv.gz', compression='gzip',index=False)